In [ ]:
import pandas as pd
import datasets
import numpy as np
import matplotlib.pyplot as plt
import datasets
import datetime

## Online Retail ###

taken from `https://archive.ics.uci.edu/dataset/352/online+retail`

In [ ]:
#import from online repos

#pip install ucimlrepo

from ucimlrepo import fetch_ucirepo  
 
online_retail = fetch_ucirepo(id=352) 
  
print('METADATA') 
print(online_retail.metadata) 
  
print("VARIABLES") 
print(online_retail.variables) 


In [ ]:
#dataset exploration

for k in online_retail.keys():
    for j in online_retail[k].keys():
        x= online_retail[k][j]
        if hasattr(x, 'shape'):
            print(k,j, x.shape)
        else: 
            print(k,j)

In [ ]:
#keep the relevant features

df = online_retail['data']['features'][['Description', 'InvoiceDate', 'Quantity']]

items = df.Description.unique()
items = sorted([str(item) for item in items])

df.loc[:,'InvoiceDate'] = pd.to_datetime(df.InvoiceDate).dt.date
start, end = min(df.InvoiceDate), max(df.InvoiceDate)

df = df.groupby(['Description', 'InvoiceDate'], group_keys=False)['Quantity'].sum()

start, end

In [ ]:
#fill the missing zeros

items = items[:items.index('add stock to allocate online orders')]

items_new = []
data_dict = {}

for item in items:
    if any(df[item] < 0):
        continue
    ts = df[item].reindex(pd.date_range(start, end),fill_value=0)
    data_dict[item] = {
        'Values' : ts.values,
        'Date' : ts.index
    }
    items_new.append(item)

len(items_new), len(items), len(data_dict[items_new[0]]['Values'])

In [ ]:
#just make a plot

item = list(data_dict.keys())[0]
ts = data_dict[item]
plt.plot(ts['Date'], ts['Values'])
ts['Values']

In [ ]:
#save it into the train?test format for datsets.data_loader

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/OnlineRetail/"

df_test = pd.DataFrame({
    'start' : [str(data_dict[item]['Date'][0]) for item in items_new],
    'target' : [list(data_dict[item]['Values']) for item in items_new],
    'feat_stat_cat' : [[i] for i in range(len(items_new))],
    'item_id' : items_new
})

df_train = pd.DataFrame({
    'start' : [str(data_dict[item]['Date'][0]) for item in items_new],
    'target' : [list(data_dict[item]['Values'])[:-28] for item in items_new],
    'feat_stat_cat' : [[i] for i in range(len(items_new))],
    'item_id' : items_new
})

df_train.to_json(path + 'train.json', orient='records')
df_test.to_json(path + 'test.json', orient='records')

pd.read_json(path + 'train.json', orient='records')

In [ ]:
#save a .csv too

pd.DataFrame(np.array([list(data_dict[item]['Values']) for item in items_new],
                      dtype=np.int64)).to_csv(path + 'data.csv', index=False)

pd.read_csv(path + 'data.csv')

In [ ]:
datasets.load_dataset(path, data_files={'train':'train.json', 'test':'test.json'})

## Auto ##

taken from `https://github.com/canerturkmen/gluon-ts/tree/intermittent-datasets/datasets/intermittent_auto`

In [ ]:
#get .json from url

import pandas as pd
import numpy as np

test_url = 'https://raw.githubusercontent.com/canerturkmen/gluon-ts/intermittent-datasets/datasets/intermittent_auto/test/data.json'
train_url = 'https://raw.githubusercontent.com/canerturkmen/gluon-ts/intermittent-datasets/datasets/intermittent_auto/train/data.json'

train = pd.read_json(train_url)
test = pd.read_json(test_url)

print(len(train.target[0]), len(test.target[0]))
test

In [ ]:
#save the .csv train and test
from datasets import Dataset

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/Auto/"

test.to_json(path + "test.json", orient='records')
train.to_json(path + "train.json", orient='records')

pd.read_json(path +  'train.json', orient='records')

In [ ]:
datasets.load_dataset(path, data_files={'train':'train.json', 'test':'test.json'})


In [ ]:
#collect it into a .csv too

data = np.empty((len(test), 24))

for i, ts in enumerate(test.target):
    data[i,] = np.array(ts, dtype=np.int64)

pd.DataFrame(data).to_csv(path + 'data.csv', index=False)

pd.read_csv(path + 'data.csv')

## RAF ##

taken from `https://github.com/canerturkmen/gluon-ts/tree/intermittent-datasets/datasets/intermittent_raf`

In [ ]:
#start importing the data

train_url = 'https://raw.githubusercontent.com/canerturkmen/gluon-ts/intermittent-datasets/datasets/intermittent_raf/train/data.json'
test_url = 'https://raw.githubusercontent.com/canerturkmen/gluon-ts/intermittent-datasets/datasets/intermittent_raf/test/data.json'

#path = "/Users/stefano.damato/switchdrive/Private/PhD/data/RAF/"

train = pd.read_json(train_url)
test = pd.read_json(test_url)

print(len(train.target[0]), len(test.target[0]))
test

In [ ]:
#save the .csv train and test

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/RAF/"

test.to_json(path + "test.json", orient='records')
train.to_json(path + "train.json", orient='records')

pd.read_json(path +  'train.json', orient='records')

In [ ]:
datasets.load_dataset(path, data_files={'train':'train.json', 'test':'test.json'})

In [ ]:
#collect it into a .csv too

data = np.empty((len(test), 84))

for i, ts in enumerate(test.target):
    data[i,] = np.array(ts, dtype=np.int64)

pd.DataFrame(data).to_csv(path + 'data.csv', index=False)

pd.read_csv(path + 'data.csv')

## carparts ##

In [ ]:
#import the data from the original .csv

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/carparts/"

pd.read_csv(path + 'carparts.csv', sep=';', index_col=0).T.to_csv(path + 'data.csv', index=False)

pd.read_csv(path + 'data.csv')

In [ ]:
#save the .csv

data = pd.read_csv(path + 'data.csv')

start = datetime.date(1998, 1, 1)

df_test = pd.DataFrame({
    'start' : [str(start) for i in range(len(data))],
    'target' : [list(data.iloc[i]) for i in range(len(data))],
    'feat_stat_cat' : [[i] for i in range(len(data))],
    'item_id' : ['T'+str(i) for i in range(len(data))],
})

df_train = pd.DataFrame({
    'start' : [str(start) for i in range(len(data))],
    'target' : [list(data.iloc[i])[:-6] for i in range(len(data))],
    'feat_stat_cat' : [[i] for i in range(len(data))],
    'item_id' : ['T'+str(i) for i in range(len(data))]
})

df_test.to_json(path + 'test.json', orient='records')
df_train.to_json(path  + 'train.json', orient ='records')

pd.read_json(path  + 'test.json', orient='records')

In [ ]:
datasets.load_dataset(path, data_files={'train':'train.json', 'test':'test.json'})


## Syph ##

In [ ]:
#import the data from the original .csv

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/Syph/"

pd.read_csv(path + 'syph.csv', sep=';', index_col=0).T.to_csv(path + 'data.csv', index=False)

data = pd.read_csv(path + 'data.csv')
data.iloc[:,:-12]

In [ ]:
#save the .json

start = datetime.datetime(2007, 1, 1)

df_test = pd.DataFrame({
    'start' : [str(start) for i in range(len(data))],
    'target' : [list(data.iloc[i]) for i in range(len(data))],
    'feat_stat_cat' : [[i] for i in range(len(data))],
    'item_id' : ['T'+str(i) for i in range(len(data))]
})

df_train = pd.DataFrame({
    'start' : [str(start) for i in range(len(data))],
    'target' : [list(data.iloc[i])[:-12] for i in range(len(data))],
    'feat_stat_cat' : [[i] for i in range(len(data))],
    'item_id' : ['T'+str(i) for i in range(len(data))]
})

df_test.to_json(path + 'test.json', orient='records')
df_train.to_json(path  + 'train.json', orient='records')

pd.read_json(path + 'test.json', orient='records')

In [ ]:
datasets.load_dataset(path =path, data_files={'train':'train.json', 'test':'test.json'})


## M5 ##

In [ ]:
#import the data from the original .csv

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/M5/"

pd.read_csv(path + 'daily_ts.csv', index_col=0).to_csv(path + 'data.csv', index=False)

data = pd.read_csv(path + 'data.csv')
data.head()

In [ ]:
#save the .json

start = datetime.datetime(2011, 1, 29)

df_test = pd.DataFrame({
    'start' : [str(start) for i in range(len(data))],
    'target' : [list(data.iloc[i]) for i in range(len(data))],
    'feat_stat_cat' : [[i] for i in range(len(data))],
    'item_id' : ['T'+str(i) for i in range(len(data))]
})

df_train = pd.DataFrame({
    'start' : [str(start) for i in range(len(data))],
    'target' : [list(data.iloc[i])[:-12] for i in range(len(data))],
    'feat_stat_cat' : [[i] for i in range(len(data))],
    'item_id' : ['T'+str(i) for i in range(len(data))]
})

df_test.to_json(path + 'test.json', orient='records')
df_train.to_json(path  + 'train.json', orient='records')

pd.read_json(path + 'test.json', orient='records')

# SOME INFOS #


In [ ]:
from metrics_scores import ts_classification

path = "/Users/stefano.damato/switchdrive/Private/PhD/data/"

dataset_names = ["OnlineRetail", "Auto", 'RAF', 'carparts', 'syph', 'M5']


for name in dataset_names:
    ds = datasets.load_dataset(path+name+'/', data_files={'train':'train.json', 'test':'test.json'})
    
    data = np.array(pd.read_csv(path+name+'/data.csv').values)
    nans = np.any(np.isnan(data), axis=1)
    #if any(nans):
        #data = data[~nans,:]
    #adi = 1/np.mean(np.where(data > .5, 1, 0), axis=1)[np.std(np.where(data > .5, 1, 0), axis=1) != 0]
    adi, cv2 = ts_classification(data)
    print(name, 
          '\nsize:', len(ds['train']), 
          '\tstart:', ds['train'][0]['start'],
          '\ttrain length:', len(ds['train'][0]['target']), 
          '\ttest length:', len(ds['test'][0]['target']),
          '\tseries w/ nans:', np.sum(nans), 
          '\tseries w/ ADI >= 1.32:', np.sum(adi>=1.32),
          '\tseries w/ ADI >= 1.32 and CV2 < 0.49:', np.sum(np.logical_and(adi >= 1.32, cv2 < .49)), '\n')

In [ ]:
def load_dataset(name, pred_length = None, idx=None):

    assert name in ["OnlineRetail", "Auto", 'RAF', 'carparts', 'syph']
    path = "../data/" + name + '/'

    if pred_length is None and idx is None:
        ds = datasets.load_dataset(path=path, data_files={'train':'train.json', 'test':'test.json'})
        ds['freq'] = '1M'
        return ds
    
    else:
        test = pd.read_json(path + 'test.json', orient='records')
        if idx is None: 
            idx = list(range(test.shape[0]))
        test, train = test.iloc[idx,:], test.iloc[idx,:]
        train.target = [ts[:-pred_length] for ts in train.target]
        return datasets.DatasetDict({'train':datasets.Dataset.from_pandas(train),
                                     'test':datasets.Dataset.from_pandas(test)})

        